In [1]:
import os
import re

import nibabel as nib
import numpy as np
import pandas as pd

In [2]:
det_path = "/home/yutongx/src_data/det_bbox/"
gt_path = "/home/yutongx/src_data/labels/"
csv_path = '/home/yutongx/src_data/info/'
save_path = "/home/yutongx/src_data/bbox_multi"
 
gt_train_path = os.path.join(gt_path, "train")
gt_val_path = os.path.join(gt_path, "val")

save_train_path = os.path.join(save_path, "train")
save_val_path = os.path.join(save_path, "val")

csv_train_path = os.path.join(csv_path, 'ribfrac-train-info.csv')
csv_val_path = os.path.join(csv_path, 'ribfrac-val-info.csv')

In [3]:
idx = lambda name: re.sub(r"\D", "", name)
get_names = lambda path: sorted([os.path.join(path, name) for name in os.listdir(path) if idx(name) != ''])

det_names = get_names(det_path)
det_train_names = sorted([name for name in det_names if int(idx(name))<=420], key=idx)
det_val_names = sorted([name for name in det_names if int(idx(name)) in range(421,501)], key=idx)
    
gt_train_names = get_names(gt_train_path)
gt_val_names = get_names(gt_val_path)

In [4]:
def masks2bboxes(masks, sub_df, border):
    instance_nums = [num for num in np.unique(masks) if num]
    bboxes = []
    
    for i in instance_nums:
        mask = (masks == i)
        if np.any(mask):
            zz, yy, xx = np.where(mask)   
            label = (sub_df.loc[sub_df['label_id'] == i])['label_code'].values[0]
            if label == -1:
                continue
            else:
                bboxes.append([(zz.max() + zz.min()) / 2.,
                           (yy.max() + yy.min()) / 2.,
                           (xx.max() + xx.min()) / 2.,
                           zz.max() - zz.min() + 1 + border,
                           yy.max() - yy.min() + 1 + border,
                           xx.max() - xx.min() + 1 + border,
                           label])
    return bboxes

def preprocessing(det_names, gt_names, save_path, csv_path, border=8, th_pos=0.6):
    save_path = save_path if os.path.exists(save_path) else os.makedirs(save_path)
    cnt_all = np.zeros(2)
    
    df = pd.read_csv(csv_path)
    
    for det_name, gt_name in zip(det_names, gt_names):
        print('====================')
        print('det_name:', det_name)
        print('gt_name:', gt_name)
        
        assert idx(det_name) == idx(gt_name), 'Index Mismatch.'
        
        det_src = np.load(det_name, allow_pickle=True)
        det_src = np.pad(det_src, ((0,0),(0,1)), 'constant')
        gt_src = (np.swapaxes(nib.load(gt_name).get_fdata(), -1, 0)).astype(np.uint8)

        sub_df = df.loc[df['public_id'] == ''.join('RibFrac' + idx(gt_name))]
        
        gt_pos = masks2bboxes(gt_src, sub_df, border=border)
        rpn_pos = []
        rpn_neg = [det_src[i, 1:] for i in range(det_src.shape[0])]

        for gt in gt_pos:
            hit = 1
            for rpn in rpn_neg:
                intsc = (rpn[3:6] + gt[3:6])/2 - np.abs(rpn[:3] - gt[:3])#calculate intersection size in 3 dims
                
                if np.all(intsc >= 1):
                    print('----------')
                    print('hit #%d:'%(hit))
                    
                    hit +=1
                    intsc_vol = np.prod(intsc)
                    iou = intsc_vol / (np.prod(rpn[3:6]) + np.prod(gt[3:6]) - intsc_vol 
                                   + np.finfo(intsc_vol.dtype).eps)#calculate iou
                    
                    print('iou:', round(iou, 4))
                    if iou > th_pos:#pos
                        rpn[-1] = gt[-1]#same with corresponding gt
                        rpn_pos.append(rpn)
        
        cnt = np.array([len(gt_pos), len(rpn_pos)])
        cnt_all += cnt
        
        print('--------------------')
        print('gt_cnt:', cnt[0])
        print('rpn_pos_cnt:', cnt[1])
        
        np.savez(os.path.join(save_path, ''.join((idx(gt_name), '_bbox.npz'))), 
                 gt_pos = np.array(gt_pos), rpn_pos=np.array(rpn_pos))
    
    print('====================')
    print('====================')
    print('gt_cnt_all:', cnt_all[0])
    print('rpn_pos_cnt_all:', cnt_all[1])

In [5]:
preprocessing(det_train_names, gt_train_names, save_train_path, csv_train_path, border=8, th_pos=0.6)

det_name: /home/yutongx/src_data/det_bbox/1_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac1-label.nii.gz
----------
hit #1:
iou: 0.8784
----------
hit #2:
iou: 0.1099
----------
hit #3:
iou: 0.109
----------
hit #1:
iou: 0.7762
----------
hit #2:
iou: 0.1002
--------------------
gt_cnt: 2
rpn_pos_cnt: 2
det_name: /home/yutongx/src_data/det_bbox/10_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac10-label.nii.gz
----------
hit #1:
iou: 0.802
----------
hit #2:
iou: 0.127
----------
hit #3:
iou: 0.0723
----------
hit #4:
iou: 0.0822
----------
hit #1:
iou: 0.0266
----------
hit #2:
iou: 0.6818
----------
hit #3:
iou: 0.0713
----------
hit #4:
iou: 0.0494
----------
hit #1:
iou: 0.021
----------
hit #2:
iou: 0.7586
----------
hit #3:
iou: 0.0077
----------
hit #1:
iou: 0.7189
----------
hit #2:
iou: 0.0989
----------
hit #3:
iou: 0.1718
----------
hit #1:
iou: 0.04
----------
hit #2:
iou: 0.8205
----------
hit #3:
iou: 0.0892
----------
hit #4:
iou: 0.

----------
hit #1:
iou: 0.4612
----------
hit #2:
iou: 0.2447
----------
hit #3:
iou: 0.0383
----------
hit #4:
iou: 0.1104
----------
hit #5:
iou: 0.1283
--------------------
gt_cnt: 1
rpn_pos_cnt: 0
det_name: /home/yutongx/src_data/det_bbox/115_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac115-label.nii.gz
----------
hit #1:
iou: 0.349
----------
hit #2:
iou: 0.2619
----------
hit #3:
iou: 0.1981
----------
hit #4:
iou: 0.0185
----------
hit #5:
iou: 0.0943
----------
hit #6:
iou: 0.0455
--------------------
gt_cnt: 1
rpn_pos_cnt: 0
det_name: /home/yutongx/src_data/det_bbox/116_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac116-label.nii.gz
----------
hit #1:
iou: 0.7136
----------
hit #2:
iou: 0.0943
----------
hit #1:
iou: 0.2696
--------------------
gt_cnt: 2
rpn_pos_cnt: 1
det_name: /home/yutongx/src_data/det_bbox/117_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac117-label.nii.gz
--------------------
gt_cnt: 0
rpn_pos_cnt

----------
hit #1:
iou: 0.8188
----------
hit #2:
iou: 0.1035
--------------------
gt_cnt: 1
rpn_pos_cnt: 1
det_name: /home/yutongx/src_data/det_bbox/129_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac129-label.nii.gz
----------
hit #1:
iou: 0.8882
----------
hit #2:
iou: 0.1875
----------
hit #1:
iou: 0.8977
----------
hit #2:
iou: 0.0962
----------
hit #3:
iou: 0.1182
----------
hit #1:
iou: 0.8063
----------
hit #2:
iou: 0.1214
----------
hit #3:
iou: 0.0146
----------
hit #1:
iou: 0.7511
----------
hit #2:
iou: 0.1142
----------
hit #3:
iou: 0.0697
----------
hit #4:
iou: 0.1484
--------------------
gt_cnt: 4
rpn_pos_cnt: 4
det_name: /home/yutongx/src_data/det_bbox/13_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac13-label.nii.gz
----------
hit #1:
iou: 0.8183
----------
hit #2:
iou: 0.0788
----------
hit #3:
iou: 0.139
----------
hit #1:
iou: 0.7489
----------
hit #1:
iou: 0.5887
--------------------
gt_cnt: 3
rpn_pos_cnt: 2
det_name: /home/yu

----------
hit #1:
iou: 0.6655
----------
hit #2:
iou: 0.0103
----------
hit #3:
iou: 0.0597
----------
hit #4:
iou: 0.1161
----------
hit #5:
iou: 0.0042
----------
hit #6:
iou: 0.2229
----------
hit #1:
iou: 0.7396
----------
hit #2:
iou: 0.1965
----------
hit #3:
iou: 0.1936
--------------------
gt_cnt: 2
rpn_pos_cnt: 2
det_name: /home/yutongx/src_data/det_bbox/140_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac140-label.nii.gz
--------------------
gt_cnt: 0
rpn_pos_cnt: 0
det_name: /home/yutongx/src_data/det_bbox/141_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac141-label.nii.gz
----------
hit #1:
iou: 0.5639
----------
hit #2:
iou: 0.2024
----------
hit #1:
iou: 0.8294
----------
hit #2:
iou: 0.1242
----------
hit #3:
iou: 0.1347
----------
hit #1:
iou: 0.0443
----------
hit #2:
iou: 0.463
----------
hit #3:
iou: 0.2231
----------
hit #1:
iou: 0.6158
----------
hit #2:
iou: 0.1109
----------
hit #1:
iou: 0.0105
----------
hit #1:
iou: 0.0155


----------
hit #1:
iou: 0.1468
----------
hit #2:
iou: 0.6709
----------
hit #3:
iou: 0.0096
----------
hit #4:
iou: 0.1229
----------
hit #1:
iou: 0.7115
----------
hit #2:
iou: 0.0804
----------
hit #3:
iou: 0.0978
----------
hit #4:
iou: 0.1074
----------
hit #5:
iou: 0.0692
----------
hit #6:
iou: 0.2103
----------
hit #7:
iou: 0.2481
----------
hit #8:
iou: 0.0432
----------
hit #1:
iou: 0.1354
----------
hit #2:
iou: 0.1312
----------
hit #3:
iou: 0.7283
----------
hit #4:
iou: 0.1389
----------
hit #1:
iou: 0.8408
----------
hit #2:
iou: 0.201
----------
hit #1:
iou: 0.7065
----------
hit #2:
iou: 0.0418
----------
hit #1:
iou: 0.013
----------
hit #2:
iou: 0.7107
----------
hit #3:
iou: 0.0902
--------------------
gt_cnt: 6
rpn_pos_cnt: 6
det_name: /home/yutongx/src_data/det_bbox/155_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac155-label.nii.gz
----------
hit #1:
iou: 0.5323
----------
hit #1:
iou: 0.715
----------
hit #2:
iou: 0.1567
----------
hit #3:
iou

...

----------
hit #1:
iou: 0.0257
----------
hit #2:
iou: 0.4312
----------
hit #3:
iou: 0.8551
----------
hit #4:
iou: 0.0381
----------
hit #5:
iou: 0.0046
----------
hit #1:
iou: 0.0504
----------
hit #2:
iou: 0.6994
----------
hit #3:
iou: 0.0518
----------
hit #4:
iou: 0.0032
----------
hit #5:
iou: 0.1432
----------
hit #6:
iou: 0.1462
----------
hit #1:
iou: 0.727
----------
hit #2:
iou: 0.1791
----------
hit #3:
iou: 0.1031
----------
hit #4:
iou: 0.0604
--------------------
gt_cnt: 3
rpn_pos_cnt: 3
det_name: /home/yutongx/src_data/det_bbox/54_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac54-label.nii.gz
----------
hit #1:
iou: 0.0635
----------
hit #2:
iou: 0.7528
----------
hit #3:
iou: 0.1228
----------
hit #1:
iou: 0.699
----------
hit #2:
iou: 0.0775
----------
hit #3:
iou: 0.0768
----------
hit #1:
iou: 0.1205
----------
hit #2:
iou: 0.5605
----------
hit #3:
iou: 0.0216
----------
hit #4:
iou: 0.5295
----------
hit #5:
iou: 0.0201
----------
hit #1:
iou:

----------
hit #1:
iou: 0.0579
----------
hit #2:
iou: 0.6244
----------
hit #3:
iou: 0.1255
----------
hit #1:
iou: 0.4705
----------
hit #2:
iou: 0.1261
----------
hit #3:
iou: 0.0222
----------
hit #4:
iou: 0.0051
----------
hit #1:
iou: 0.0518
----------
hit #2:
iou: 0.317
----------
hit #3:
iou: 0.0714
----------
hit #4:
iou: 0.0085
----------
hit #1:
iou: 0.0524
----------
hit #2:
iou: 0.7197
----------
hit #3:
iou: 0.0016
----------
hit #4:
iou: 0.0976
----------
hit #1:
iou: 0.5824
----------
hit #2:
iou: 0.0163
----------
hit #3:
iou: 0.2539
----------
hit #4:
iou: 0.0322
--------------------
gt_cnt: 5
rpn_pos_cnt: 2
det_name: /home/yutongx/src_data/det_bbox/66_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac66-label.nii.gz
----------
hit #1:
iou: 0.6459
----------
hit #2:
iou: 0.0322
----------
hit #3:
iou: 0.1319
----------
hit #1:
iou: 0.7913
----------
hit #2:
iou: 0.1718
----------
hit #3:
iou: 0.0486
--------------------
gt_cnt: 2
rpn_pos_cnt: 2
det_nam

----------
hit #1:
iou: 0.8205
----------
hit #2:
iou: 0.0194
----------
hit #3:
iou: 0.0058
----------
hit #4:
iou: 0.0095
----------
hit #1:
iou: 0.86
----------
hit #2:
iou: 0.0213
----------
hit #3:
iou: 0.1529
----------
hit #4:
iou: 0.0815
----------
hit #1:
iou: 0.0056
----------
hit #2:
iou: 0.8089
----------
hit #1:
iou: 0.0201
----------
hit #2:
iou: 0.7127
----------
hit #3:
iou: 0.1411
----------
hit #1:
iou: 0.8416
----------
hit #2:
iou: 0.1647
----------
hit #1:
iou: 0.6453
----------
hit #2:
iou: 0.0145
----------
hit #3:
iou: 0.049
--------------------
gt_cnt: 6
rpn_pos_cnt: 6
det_name: /home/yutongx/src_data/det_bbox/8_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac8-label.nii.gz
----------
hit #1:
iou: 0.4488
----------
hit #2:
iou: 0.0505
----------
hit #3:
iou: 0.0228
----------
hit #1:
iou: 0.3408
----------
hit #2:
iou: 0.1217
----------
hit #3:
iou: 0.0925
----------
hit #1:
iou: 0.0177
----------
hit #2:
iou: 0.755
----------
hit #3:
iou: 0.0

----------
hit #1:
iou: 0.0101
----------
hit #2:
iou: 0.767
----------
hit #3:
iou: 0.1101
----------
hit #4:
iou: 0.1587
----------
hit #1:
iou: 0.6916
----------
hit #2:
iou: 0.0186
----------
hit #3:
iou: 0.1118
----------
hit #4:
iou: 0.0993
----------
hit #5:
iou: 0.0585
----------
hit #1:
iou: 0.803
----------
hit #2:
iou: 0.1519
----------
hit #3:
iou: 0.1073
----------
hit #1:
iou: 0.6752
----------
hit #2:
iou: 0.0646
----------
hit #1:
iou: 0.5053
----------
hit #2:
iou: 0.5401
----------
hit #1:
iou: 0.0188
----------
hit #2:
iou: 0.7153
----------
hit #3:
iou: 0.0404
----------
hit #1:
iou: 0.6982
----------
hit #2:
iou: 0.1513
--------------------
gt_cnt: 7
rpn_pos_cnt: 6
det_name: /home/yutongx/src_data/det_bbox/9_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/train/RibFrac9-label.nii.gz
----------
hit #1:
iou: 0.0259
----------
hit #2:
iou: 0.1136
----------
hit #3:
iou: 0.5624
----------
hit #4:
iou: 0.0529
----------
hit #5:
iou: 0.1763
----------
hit #6:
iou: 0

--------------------
gt_cnt: 0
rpn_pos_cnt: 0
gt_cnt_all: 1655.0
rpn_pos_cnt_all: 1186.0


In [6]:
preprocessing(det_val_names, gt_val_names, save_val_path, csv_val_path, border=8, th_pos=0.6)

det_name: /home/yutongx/src_data/det_bbox/421_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/val/RibFrac421-label.nii.gz
----------
hit #1:
iou: 0.659
----------
hit #2:
iou: 0.0895
----------
hit #3:
iou: 0.0481
--------------------
gt_cnt: 1
rpn_pos_cnt: 1
det_name: /home/yutongx/src_data/det_bbox/422_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/val/RibFrac422-label.nii.gz
----------
hit #1:
iou: 0.6613
----------
hit #1:
iou: 0.0377
----------
hit #2:
iou: 0.9054
----------
hit #1:
iou: 0.0308
----------
hit #2:
iou: 0.6254
----------
hit #3:
iou: 0.082
----------
hit #4:
iou: 0.0721
----------
hit #1:
iou: 0.6802
----------
hit #2:
iou: 0.1043
----------
hit #3:
iou: 0.055
----------
hit #4:
iou: 0.1854
----------
hit #1:
iou: 0.7585
----------
hit #2:
iou: 0.1344
--------------------
gt_cnt: 5
rpn_pos_cnt: 5
det_name: /home/yutongx/src_data/det_bbox/423_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/val/RibFrac423-label.nii.gz
----------
hit #1:
iou: 0.1415
---

--------------------
gt_cnt: 0
rpn_pos_cnt: 0
det_name: /home/yutongx/src_data/det_bbox/440_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/val/RibFrac440-label.nii.gz
----------
hit #1:
iou: 0.1131
--------------------
gt_cnt: 1
rpn_pos_cnt: 0
det_name: /home/yutongx/src_data/det_bbox/441_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/val/RibFrac441-label.nii.gz
----------
hit #1:
iou: 0.7968
----------
hit #1:
iou: 0.9252
----------
hit #1:
iou: 0.7592
----------
hit #2:
iou: 0.0774
----------
hit #1:
iou: 0.6207
----------
hit #2:
iou: 0.1095
----------
hit #3:
iou: 0.0493
----------
hit #1:
iou: 0.6831
----------
hit #1:
iou: 0.6214
----------
hit #2:
iou: 0.0984
----------
hit #3:
iou: 0.1161
----------
hit #1:
iou: 0.6505
----------
hit #2:
iou: 0.0246
----------
hit #3:
iou: 0.0529
----------
hit #1:
iou: 0.5081
----------
hit #2:
iou: 0.3844
----------
hit #1:
iou: 0.2488
----------
hit #2:
iou: 0.0348
----------
hit #3:
iou: 0.2707
----------
hit #4:
iou: 0.1055
---

--------------------
gt_cnt: 0
rpn_pos_cnt: 0
det_name: /home/yutongx/src_data/det_bbox/462_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/val/RibFrac462-label.nii.gz
--------------------
gt_cnt: 0
rpn_pos_cnt: 0
det_name: /home/yutongx/src_data/det_bbox/463_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/val/RibFrac463-label.nii.gz
----------
hit #1:
iou: 0.0943
----------
hit #2:
iou: 0.3473
----------
hit #3:
iou: 0.1607
----------
hit #4:
iou: 0.0294
----------
hit #1:
iou: 0.485
----------
hit #2:
iou: 0.0758
----------
hit #3:
iou: 0.0729
----------
hit #4:
iou: 0.219
----------
hit #5:
iou: 0.0363
----------
hit #6:
iou: 0.1585
----------
hit #1:
iou: 0.2127
----------
hit #2:
iou: 0.5023
----------
hit #3:
iou: 0.0535
----------
hit #4:
iou: 0.0145
----------
hit #1:
iou: 0.4856
----------
hit #2:
iou: 0.058
----------
hit #3:
iou: 0.1061
----------
hit #4:
iou: 0.1378
----------
hit #5:
iou: 0.1669
----------
hit #1:
iou: 0.5858
----------
hit #2:
iou: 0.1285
------

----------
hit #1:
iou: 0.6162
----------
hit #2:
iou: 0.1868
----------
hit #3:
iou: 0.3703
----------
hit #1:
iou: 0.657
----------
hit #1:
iou: 0.869
----------
hit #2:
iou: 0.0913
----------
hit #1:
iou: 0.5102
--------------------
gt_cnt: 4
rpn_pos_cnt: 3
det_name: /home/yutongx/src_data/det_bbox/482_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/val/RibFrac482-label.nii.gz
----------
hit #1:
iou: 0.733
----------
hit #2:
iou: 0.0144
----------
hit #3:
iou: 0.1106
----------
hit #4:
iou: 0.0779
----------
hit #5:
iou: 0.0635
----------
hit #1:
iou: 0.0312
----------
hit #2:
iou: 0.6414
----------
hit #3:
iou: 0.1946
----------
hit #1:
iou: 0.6518
----------
hit #2:
iou: 0.0556
----------
hit #3:
iou: 0.1383
----------
hit #4:
iou: 0.0419
----------
hit #1:
iou: 0.1079
----------
hit #2:
iou: 0.73
----------
hit #3:
iou: 0.0907
----------
hit #4:
iou: 0.1298
----------
hit #1:
iou: 0.7182
----------
hit #2:
iou: 0.052
----------
hit #3:
iou: 0.1617
----------
hit #1:
iou: 0.0

----------
hit #1:
iou: 0.4641
----------
hit #2:
iou: 0.2024
----------
hit #1:
iou: 0.8428
----------
hit #2:
iou: 0.1208
----------
hit #3:
iou: 0.0224
----------
hit #1:
iou: 0.0144
----------
hit #2:
iou: 0.6935
----------
hit #3:
iou: 0.092
--------------------
gt_cnt: 3
rpn_pos_cnt: 2
det_name: /home/yutongx/src_data/det_bbox/498_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/val/RibFrac498-label.nii.gz
----------
hit #1:
iou: 0.7559
----------
hit #2:
iou: 0.1292
----------
hit #3:
iou: 0.0687
----------
hit #4:
iou: 0.0725
----------
hit #1:
iou: 0.8406
----------
hit #2:
iou: 0.1089
----------
hit #3:
iou: 0.2386
----------
hit #4:
iou: 0.0071
----------
hit #1:
iou: 0.629
----------
hit #2:
iou: 0.0702
----------
hit #3:
iou: 0.0386
----------
hit #4:
iou: 0.086
--------------------
gt_cnt: 3
rpn_pos_cnt: 3
det_name: /home/yutongx/src_data/det_bbox/499_rpns_list.npy
gt_name: /home/yutongx/src_data/labels/val/RibFrac499-label.nii.gz
----------
hit #1:
iou: 0.3864
------